In [5]:
# ----- Cell 2: Config -----
import os, random
from glob import glob

DATA_DIR = "C:\\Users\\laksh\\Desktop\\Visual Code\\Aslinakli\\data\\audio"
SUBSET_SIZE = 5000      # instead of 500
TRAIN_SPLIT = 0.85
SAMPLE_RATE = 16000


In [6]:
# ----- Cell 3: Subset + Split ----

# Match real and fake files under any language folder
all_files = glob(os.path.join(DATA_DIR, "*", "real", "*.wav")) + \
            glob(os.path.join(DATA_DIR, "*", "fake", "*.wav"))

# Shuffle and select subset
random.shuffle(all_files)
subset_files = all_files[:SUBSET_SIZE]

# Train-test split
split_index = int(len(subset_files) * TRAIN_SPLIT)
train_files = subset_files[:split_index]
test_files  = subset_files[split_index:]

# ----- Class balance check -----
num_real = sum("real" in f for f in subset_files)
num_fake = sum("fake" in f for f in subset_files)
print(f"Subset size: {len(subset_files)} | Real: {num_real}, Fake: {num_fake}")


Subset size: 5000 | Real: 2506, Fake: 2494


In [7]:
#Cell 4

import torchaudio
from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self, files, sr=SAMPLE_RATE):
        self.files = files
        self.sr    = sr

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path  = self.files[idx]
        label = 1 if "/real/" in path else 0
        sig, _ = torchaudio.load(path)
        return sig[0], label

# drop_last=True prevents small 1-sample batches (which break ECAPA)
train_loader = DataLoader(AudioDataset(train_files), batch_size=8, shuffle=True, drop_last=True)
test_loader  = DataLoader(AudioDataset(test_files),  batch_size=8, drop_last=True)


In [8]:
# ----- Cell 5
import torch
import torch.nn as nn
from speechbrain.inference import EncoderClassifier

sb_model = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

# Replace last classifier layer with 2-class layer
in_dim = sb_model.mods.classifier.weight.shape[1]
sb_model.mods.classifier = nn.Linear(in_dim, 2)

device = "cuda" if torch.cuda.is_available() else "cpu"
sb_model = sb_model.to(device)

#  🔧 Updated optimizer with lower learning rate
optimizer = torch.optim.Adam(sb_model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()


c:\Users\laksh\Desktop\Visual Code\Aslinakli\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\laksh\Desktop\Visual Code\Aslinakli\venv\lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
c:\Users\laksh\Desktop\Visual Code\Aslinakli\venv\lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio._backe

In [9]:
from tqdm import tqdm  # Add this import at the top of your notebook if not already

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(sb_model.parameters(), lr=1e-4)

epochs = 15
for epoch in range(epochs):
    sb_model.train()
    running_loss = 0.0

    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    for wav, y in tqdm(train_loader, desc=f"Training", leave=False):
        wav = wav.to(device)
        y   = y.to(device)

        out = sb_model(wav)

        if isinstance(out, tuple):
            logits = out[0]
        else:
            logits = out

        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"✅ Epoch {epoch+1} | Train Loss: {running_loss/len(train_loader):.4f}")



🔁 Epoch 1/15


Training:   0%|          | 0/531 [00:00<?, ?it/s]c:\Users\laksh\Desktop\Visual Code\Aslinakli\venv\lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
CategoricalEncoder.expect_len was never called: assuming category count of 7205 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct size. If you are sure this does not apply to you, use `.ignore_len`.


✅ Epoch 1 | Train Loss: 5.1619

🔁 Epoch 2/15


✅ Epoch 2 | Train Loss: 2.9960

🔁 Epoch 3/15


✅ Epoch 3 | Train Loss: 1.9082

🔁 Epoch 4/15


...

✅ Epoch 11 | Train Loss: 0.2762

🔁 Epoch 12/15


✅ Epoch 12 | Train Loss: 0.2553

🔁 Epoch 13/15


✅ Epoch 13 | Train Loss: 0.2353

🔁 Epoch 14/15


✅ Epoch 14 | Train Loss: 0.2192

🔁 Epoch 15/15


✅ Epoch 15 | Train Loss: 0.2017


In [10]:
sb_model.eval()
correct = 0; total = 0

with torch.no_grad():
    for wav, y in test_loader:
        wav = wav.to(device)
        y   = y.to(device)

        out = sb_model(wav)
        logits = out[0] if isinstance(out, tuple) else out
        preds = logits.argmax(dim=1)

        correct += (preds == y).sum().item()
        total   += y.size(0)

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 99.06%


In [ ]:
import torch
from datetime import datetime

model_path = "models/Rawnetlite_Model_Output.pt"
torch.save(sb_model, model_path)
print("Model saved to:", model_path)

log_path = "utils/Audio_training_logs.csv"
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
log_row = f"{timestamp},5,{test_acc:.2f}\n"

with open(log_path, "a") as log_file:
    log_file.write(log_row)

print("Log entry added:", log_row)


Model saved to: Rawnetlite_Model_Output.pt
Log entry added: 2025-08-25 00:01:47,5,99.06

